In [1]:
import pandas as pd 
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [2]:
# dataset
df = pd.read_csv('WELFake_Dataset.csv')

In [3]:
df = df.dropna()

In [4]:
# stemmer
ps_ = PorterStemmer()

In [5]:
# preprocessing data
corpus = []
for i in range(0, len(df)):
    print(f"Title No: {i}", "\n")
    review = re.sub('[^a-zA-Z]', ' ', df['title'].iloc[i])
    review = review.lower()
    review = review.split()
    review = [ps_.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    
df['cleaned_text'] = corpus

Title No: 0 

Title No: 1 

Title No: 2 

Title No: 3 

Title No: 4 

Title No: 5 

Title No: 6 

Title No: 7 

Title No: 8 

Title No: 9 

Title No: 10 

Title No: 11 

Title No: 12 

Title No: 13 

Title No: 14 

Title No: 15 

Title No: 16 

Title No: 17 

Title No: 18 

Title No: 19 

Title No: 20 

Title No: 21 

Title No: 22 

Title No: 23 

Title No: 24 

Title No: 25 

Title No: 26 

Title No: 27 

Title No: 28 

Title No: 29 

Title No: 30 

Title No: 31 

Title No: 32 

Title No: 33 

Title No: 34 

Title No: 35 

Title No: 36 

Title No: 37 

Title No: 38 

Title No: 39 

Title No: 40 

Title No: 41 

Title No: 42 

Title No: 43 

Title No: 44 

Title No: 45 

Title No: 46 

Title No: 47 

Title No: 48 

Title No: 49 

Title No: 50 

Title No: 51 

Title No: 52 

Title No: 53 

Title No: 54 

Title No: 55 

Title No: 56 

Title No: 57 

Title No: 58 

Title No: 59 

Title No: 60 

Title No: 61 

Title No: 62 

Title No: 63 

Title No: 64 

Title No: 65 

Title No: 66 

Title

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

with open("tfidf_vectorizer.pkl", "rb") as file:
    vectorizer = pickle.load(file)

# Transform the cleaned text into vectors
X_custom = vectorizer.transform(df['cleaned_text'])

In [7]:
X_custom

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 615616 stored elements and shape (71537, 10000)>

In [8]:
# Preprocessing Data

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Splitting the dataset
y = df['label']  
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_text'], y, test_size=0.2, random_state=42)

max_words = 10000  
max_len = 85    
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')


2024-12-10 23:03:01.285745: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733851981.336429   63046 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733851981.350116   63046 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-10 23:03:01.467623: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# LSTM Model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


/home/anson/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1733852006.377050   63046 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3620 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 6GB Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
history = model.fit(
    X_train_padded, y_train,
    validation_data=(X_test_padded, y_test),
    epochs=5,
    batch_size=32,
    verbose=1
)


Epoch 1/5


I0000 00:00:1733852019.826676   66306 cuda_dnn.cc:529] Loaded cuDNN version 90300


1789/1789 ━━━━━━━━━━━━━━━━━━━━ 72s 36ms/step - accuracy: 0.8389 - loss: 0.3420 - val_accuracy: 0.9119 - val_loss: 0.2116
Epoch 2/5
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 64s 36ms/step - accuracy: 0.9380 - loss: 0.1615 - val_accuracy: 0.9135 - val_loss: 0.2159
Epoch 3/5
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 63s 35ms/step - accuracy: 0.9550 - loss: 0.1157 - val_accuracy: 0.9070 - val_loss: 0.2402
Epoch 4/5
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 64s 36ms/step - accuracy: 0.9679 - loss: 0.0830 - val_accuracy: 0.9126 - val_loss: 0.2787
Epoch 5/5
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 65s 36ms/step - accuracy: 0.9752 - loss: 0.0619 - val_accuracy: 0.9090 - val_loss: 0.3660


In [11]:
model.save("fake_news_lstm_model.h5")


In [12]:
with open("tokenizer.pkl", "wb") as file:
    pickle.dump(tokenizer, file)

In [13]:
#Predictions
from tensorflow.keras.models import load_model

model = load_model("fake_news_lstm_model.h5")
with open("tokenizer.pkl", "rb") as file:
    tokenizer = pickle.load(file)

In [14]:
test_data = pd.read_csv('test.csv')

In [18]:
test_data.isnull().sum()

id        0
title     0
author    0
text      0
dtype: int64

In [17]:
test_data = test_data.dropna()

In [19]:
test_data['combined_text'] = test_data['title'] + " " + test_data['text']

/tmp/ipykernel_63046/1451563311.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['combined_text'] = test_data['title'] + " " + test_data['text']


In [20]:
# Preprocess and clean the text
test_data['cleaned_text'] = [
    ' '.join([ps_.stem(word) for word in re.sub('[^a-zA-Z]', ' ', row).lower().split() if word not in stopwords.words('english')])
    for row in test_data['combined_text']
]

/tmp/ipykernel_63046/3530871742.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['cleaned_text'] = [


In [21]:
test_seq = tokenizer.texts_to_sequences(test_data['cleaned_text'])


test_padded = pad_sequences(test_seq, maxlen=max_len, padding='post', truncating='post')


In [22]:
# Make predictions
test_predictions = (model.predict(test_padded) > 0.5).astype("int32")

test_data['predicted_label'] = test_predictions


143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step


/tmp/ipykernel_63046/3639861643.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predicted_label'] = test_predictions


In [23]:
test_data.to_csv("test_predictions.csv", index=False)

